协作型过滤：我们知道，要想了解商品、影片或娱乐性网站的推荐信息，最没有技术含量的方法莫过于向朋友们询问。我们也知道，这其中有一部分人的品味会比其他人的高一些，通过观察这些人是否通常也和我们一样喜欢同样的东西，可以逐渐对这些情况有所了解。不过随着选择越来越多，要想通过询问一小群人来确定我们想要的东西，将会变得越来越不切实际，因为他们可能并不了解所有的选择。这就是人们为什么要发展出一套被称为协作型过滤的技术。

一个协作型过滤算法通常的做法是对一大群人进行搜索，并从中找出与我们品味相近的一小群人。算法会对这些人所偏爱的其他内容进行考察，并将它们组合起来构造出一个经过排名的推荐列表，有许多不同的方法可以帮助我们确定哪些人与自己的品味相近，并将他们的选择组合成列表。


In [2]:
from importlib import reload
import recommendations
from recommendations import critics

In [3]:
critics['Lisa Rose']['Lady in the Water']

2.5

In [4]:
critics['Toby']['Snakes on a Plane'] = 4.5

In [5]:
critics['Toby']

{'Snakes on a Plane': 4.5, 'Superman Returns': 4.0, 'You, Me and Dupree': 1.0}

尽管可以将相当数量的人员偏好信息置于字典内（即内存中），但对于一个规模巨大的数据集而言，也许我们还是会希望将其存入数据库中。

寻找相近的用户

计算相似度评价值的方法：欧几里得距离和皮尔逊相关度

欧几里得距离评价

In [6]:
from math import sqrt

In [7]:
sqrt(pow(4.5 - 4, 2) + pow(1 - 2, 2))

1.118033988749895

上述算式可以计算出距离值，偏好越相似的人，其距离就越短。此外，我们还需要一个函数，来对偏好越相近的情况给出越大的值。为此，我们可以将函数值加1（这样可以避免遇到被0整除的错误了），并取其倒数。

In [8]:
1 / (1 + sqrt(pow(4.5 - 4, 2) + pow(1 - 2, 2)))

0.4721359549995794

这一新的函数总是返回介于0到1之间的值，返回1则表示两人具有一样的偏好。

In [10]:
from recommendations import sim_distance
sim_distance(critics, 'Lisa Rose', 'Gene Seymour')

0.29429805508554946

上述执行过程给出了Lisa Rose和Gene Seymour之间的相似度评价。请用其他人的名字试试，看看我们是否能够找到或多或少具有一定共性的人。

In [11]:
p=[]
for p1 in critics.keys():
    for p2 in critics.keys():
        if p1 != p2 and p2 not in p:
            print("The distance between " + p1 + ' and ' + p2 +  ' is: ' + str(sim_distance(critics, p1,p2)))
        p.append(p1)

The distance between Jack Matthews and Claudia Puig is: 0.32037724101704074
The distance between Jack Matthews and Toby is: 0.2674788903885893
The distance between Jack Matthews and Mick LaSalle is: 0.2857142857142857
The distance between Jack Matthews and Gene Seymour is: 0.6666666666666666
The distance between Jack Matthews and Michael Phillips is: 0.29429805508554946
The distance between Jack Matthews and Lisa Rose is: 0.3405424265831667
The distance between Claudia Puig and Toby is: 0.3567891723253309
The distance between Claudia Puig and Mick LaSalle is: 0.31451985913875646
The distance between Claudia Puig and Gene Seymour is: 0.28172904669025317
The distance between Claudia Puig and Michael Phillips is: 0.585786437626905
The distance between Claudia Puig and Lisa Rose is: 0.38742588672279304
The distance between Toby and Mick LaSalle is: 0.4
The distance between Toby and Gene Seymour is: 0.25824569976124334
The distance between Toby and Michael Phillips is: 0.38742588672279304
T

皮尔逊相关度评价（判断两组数的线性关系程度）

除了欧几里得距离，还有一种更复杂一些的方法可以用来判断人们兴趣的相似度，那就是皮尔逊相关系数。该相关系数是判断两组数据与某一直线拟合程度的一种度量。对应的公式要比欧几里得距离评价的计算公式要复杂，但是它在数据不是很规范（normalized）的时候（比如，影评者对影片的评价总是相对于平均水平偏离很大时），会倾向于给出更好的结果。在采用皮尔逊方法进行评价时，它可以修正“夸大分值”的情况。如果某人总是倾向于给出比另一个人更高的比值，而二者的分值之差又始终保持一致，则他们依然可能会存在很好的相关性。此前提到的欧几里得距离评价方法，会因为一个人的评价始终比另一个人的更为“严格”（从而导致评价始终相对偏低），而得出两者不相近的结论，即使他们的品味很相似也是如此，而这一行为是否就是我们想要的结果 ，则取决于具体的应用场景。

假设有两个变量X、Y，那么两变量间的皮尔逊相关系数可通过以下公式计算：

公式一：
<img src="imgs/皮尔逊相关系数1.gif">

公式二：
<img src="imgs/皮尔逊相关系数2.gif">

公式三：
<img src="imgs/皮尔逊相关系数3.gif">

公式四：
<img src="imgs/皮尔逊相关系数4.gif">

以上列出的四个公式等价，其中E是数学期望，cov表示协方差，N表示变量取值的个数。
 
皮尔逊相关度评价算法首先会找出两位评论者都曾评论过的物品，然后计算两者的评分总和与平方和，并求得评分的乘积之和。利用上面的公式四计算出皮尔逊相关系数。

In [27]:
reload(recommendations)
from recommendations import sim_pearson

In [28]:
sim_pearson(critics, 'Lisa Rose', 'Gene Seymour')

18.0
19.5
55.0
69.75
59.5


0.39605901719066977

该函数返回一个介于-1与1之间的数值。值为1则表明两个人对每一样物品均有着完全一致的评价。与距离度量法不同，此处我们无须为达到正确的比率而对这一数值进行变换。

扩展：按照高中数学水平来理解, 皮尔逊相关系数, 可以看做将两组数据首先做Z分数处理之后, 然后两组数据的乘积和除以样本数，Z分数一般代表正态分布中, 数据偏离中心点的距离.等于变量减掉平均数再除以标准差.z分数（z-score）,也叫标准分数（standard score）是一个数与平均数的差再除以标准差的过程。

z分数可以回答这样一个问题："一个给定分数距离平均数多少个标准差?"在平均数之上的分数会得到一个正的标准分数，在平均数之下的分数会得到一个负的标准分数。　z分数是一种可以看出某分数在分布中相对位置的方法。z分数能够真实的反应一个分数距离平均数的相对标准距离。如果我们把每一个分数都转换成z分数，那么每一个z分数会以标准差为单位表示一个具体分数到平均数的距离或离差。将成正态分布的数据中的原始分数转换为z分数，我们就可以通过查阅z分数在正态曲线下面积的表格来得知平均数与z分数之间的面积，进而得知原始分数在数据集合中的百分等级。一个数列的各z分数的平方和等于该数列数据的个数，并且z分数的标准差和方差都为1.平均数为0.

<img src="imgs/Z分数.png">

应该选用哪一种相似性度量方法，完全取决于具体的应用。如果你想看看哪种方法能够获得更好的实际效果，皮尔逊、欧几里得距离或者其他方法，都是值得一试的。

为评论者打分

调用该方法并传入自己的姓名，将得到一个有关影评者及其相似度评价的列表。

In [32]:
reload(recommendations)
recommendations.topMatches(critics, 'Toby', n=3)

[(0.9912407071619299, 'Lisa Rose'),
 (0.9244734516419049, 'Mick LaSalle'),
 (0.8934051474415647, 'Claudia Puig')]

根据返回的结果我们了解到，应当阅读Lisa Rose所撰写的评论，因为她的品味与我们的很相近。如果你看过这些电影，也不妨将自己的偏好信息加入字典中，然后看看谁是你最喜欢的评论者。

推荐物品

In [35]:
reload(recommendations)
recommendations.getRecommendations(critics, 'Toby')

[(3.3477895267131013, 'The Night Listener'),
 (2.832549918264162, 'Lady in the Water'),
 (2.5309807037655645, 'Just My Luck')]

In [36]:
recommendations.getRecommendations(critics, 'Toby', similarity=sim_distance)

[(3.5531083380650106, 'The Night Listener'),
 (2.7785840038149234, 'Lady in the Water'),
 (2.422482042361917, 'Just My Luck')]

匹配商品

现在我们已经知道了如何为指定人员寻找品味相近者，以及如何向其推荐商品的方法，现在来了解哪些商品是彼此相近的。在这种情况下，我们可以通过查看哪些人喜欢某一特定物品，以及这些人喜欢哪些其他物品来决定相似度。事实上这和我们此前用来决定人与人之间相似度的方法是一样的，只需将人员与物品兑换即可。

In [38]:
reload(recommendations)
movies = recommendations.transformPrefs(critics)

In [39]:
movies

{'Just My Luck': {'Claudia Puig': 3.0,
  'Gene Seymour': 1.5,
  'Lisa Rose': 3.0,
  'Mick LaSalle': 2.0},
 'Lady in the Water': {'Gene Seymour': 3.0,
  'Jack Matthews': 3.0,
  'Lisa Rose': 2.5,
  'Michael Phillips': 2.5,
  'Mick LaSalle': 3.0},
 'Snakes on a Plane': {'Claudia Puig': 3.5,
  'Gene Seymour': 3.5,
  'Jack Matthews': 4.0,
  'Lisa Rose': 3.5,
  'Michael Phillips': 3.0,
  'Mick LaSalle': 4.0,
  'Toby': 4.5},
 'Superman Returns': {'Claudia Puig': 4.0,
  'Gene Seymour': 5.0,
  'Jack Matthews': 5.0,
  'Lisa Rose': 3.5,
  'Michael Phillips': 3.5,
  'Mick LaSalle': 3.0,
  'Toby': 4.0},
 'The Night Listener': {'Claudia Puig': 4.5,
  'Gene Seymour': 3.0,
  'Jack Matthews': 3.0,
  'Lisa Rose': 3.0,
  'Michael Phillips': 4.5,
  'Mick LaSalle': 3.0},
 'You, Me and Dupree': {'Claudia Puig': 2.5,
  'Gene Seymour': 3.5,
  'Jack Matthews': 3.5,
  'Lisa Rose': 2.5,
  'Mick LaSalle': 2.0,
  'Toby': 1.0}}

In [40]:
recommendations.topMatches(movies, 'Superman Returns')

[(0.6579516949597695, 'You, Me and Dupree'),
 (0.4879500364742689, 'Lady in the Water'),
 (0.11180339887498941, 'Snakes on a Plane'),
 (-0.23145502494313785, 'The Night Listener'),
 (-0.42289003161103106, 'Just My Luck')]

请注意，在本例中实际存在着一些相关评价值为负的情况，这表明那些喜欢影片Superman Returns的人存在不喜欢Just My Luck的倾向。

上面我们示范了为某部影片提供相关影片的推荐，不仅如此，我们甚至还可以为影片推荐评论者。例如，也许我们正在考虑邀请谁和自己一起参加某部影片的首映式。

In [41]:
recommendations.getRecommendations(movies, 'Just My Luck')

[(4.5, 'Michael Phillips'), (3.0, 'Jack Matthews')]

将人和物对调并不总是会得到有价值的结果，但是大多数情况下，这将有助于我们做出有意义的对比。为了向不同的个体推荐商品，在线零售商可能会收集人们的购买历史。将商品与人进行对调--正如我们此前所作的那样--可以令零售商找到购买某些商品的潜在客户。这对于他们为了清仓处理某些商品而在市场营销投入方面制定的规划，也许是很有助益的。这种做法的另一个潜在用途是，在专门推荐链接的网站上，这样做可以确保新出现的链接，能够被那些最有可能对它产生兴趣的网站用户找到。

构建一个基于del.icio.us的链接推荐系统